In [1]:
# Installing Spark 3.0.1 with dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
print('JDK 8 installed')
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
print('Spark 3.0.1 installer downloaded')
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
print('Spark 3.0.1 installed')
!pip install -q findspark
print('findspark installed')
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"
print('Environment Variables Set')
import findspark
findspark.init()
print('Install Complete')

JDK 8 installed
Spark 3.0.1 installer downloaded
Spark 3.0.1 installed
findspark installed
Environment Variables Set
Install Complete
